In [64]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import img_to_array
from pipeline.nn.conv import LeNet
from imutils import paths
import numpy as np
from tensorflow.keras.utils import to_categorical
import argparse
import imutils
import cv2
import os


In [65]:
data_set = "dataset/SMILEsmileD/SMILEs"
output = "output"
# initialize the list of data and labels
data = []
labels = []

In [66]:
# loop over the input images
for imagePath in sorted(list(paths.list_images(data_set))):
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = imutils.resize(image, width=28)
    image = img_to_array(image)
    data.append(image)

    # extract the class label from the image path and update the label list
    label = imagePath.split(os.path.sep)[-3]
    label = "smiling" if label == "positives" else "not_smiling"
    labels.append(label)

count = 0
for i in range(len(labels)):
    if labels[i] == "smiling":
        count += 1

print(count)

3690


In [67]:
len(labels)
labels

['not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_smiling',
 'not_sm

In [68]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [69]:
labels

array(['not_smiling', 'not_smiling', 'not_smiling', ..., 'smiling',
       'smiling', 'smiling'], dtype='<U11')

In [70]:
# convert the labels from integers to vectors
le = LabelEncoder().fit(labels)
labels = to_categorical(le.transform(labels), 2)

# account for skew in the labeled data
classTotals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals

In [71]:
classTotals

array([9475., 3690.], dtype=float32)

In [72]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)

In [73]:
print(len(trainX), len(trainY), len(testX), len(testY))

10532 10532 2633 2633


In [74]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=1, classes=2)
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

[INFO] compiling model...


In [76]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=15, verbose=1)

[INFO] training network...
Epoch 1/15


2021-11-03 22:16:42.730959: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


165/165 [==============================] - 15s 85ms/step - loss: 0.4197 - accuracy: 0.8080 - val_loss: 0.2990 - val_accuracy: 0.8754
Epoch 2/15
165/165 [==============================] - 15s 90ms/step - loss: 0.2785 - accuracy: 0.8877 - val_loss: 0.2755 - val_accuracy: 0.8921
Epoch 3/15
165/165 [==============================] - 15s 94ms/step - loss: 0.2584 - accuracy: 0.8981 - val_loss: 0.2537 - val_accuracy: 0.9001
Epoch 4/15
165/165 [==============================] - 17s 100ms/step - loss: 0.2401 - accuracy: 0.9071 - val_loss: 0.2509 - val_accuracy: 0.9054
Epoch 5/15
165/165 [==============================] - 17s 105ms/step - loss: 0.2214 - accuracy: 0.9128 - val_loss: 0.2485 - val_accuracy: 0.8994
Epoch 6/15
165/165 [==============================] - 20s 121ms/step - loss: 0.2098 - accuracy: 0.9202 - val_loss: 0.2554 - val_accuracy: 0.8967
Epoch 7/15
165/165 [==============================] - 18s 109ms/step - loss: 0.2079 - accuracy: 0.9194 - val_loss: 0.2375 - val_accuracy: 0.9115

In [77]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

 not_smiling       0.93      0.95      0.94      1895
     smiling       0.87      0.82      0.84       738

    accuracy                           0.91      2633
   macro avg       0.90      0.89      0.89      2633
weighted avg       0.91      0.91      0.91      2633



In [79]:
# save the model to disk
model.save(output)

2021-11-03 22:23:16.551282: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: output/assets
